In [182]:
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='ticks', context='talk')

In [2]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'
allFiles=glob.glob(path+"*.csv")
frame=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file, index_col=[0])
    listed.append(df)
frame = pd.concat(listed)

In [3]:
huc12=gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp') #reads in the hydraulic basin units and adds an area column
huc12= huc12.to_crs({'init': 'epsg:3732'})


In [4]:
frame['HUC_12']=frame.HUC_12.astype(str)

In [5]:
CaptainHucstable = huc12.merge(frame, on='HUC_12' )

In [71]:
pruned=CaptainHucstable[CaptainHucstable['prob']>0]

In [237]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
%matplotlib nbagg

square2 = folium.Map(location=wyo_coord, zoom_start=7)
formations = sorted(pruned['formation'].unique().tolist())
elements = sorted(pruned['element'].unique().tolist())
#ppm = sorted(pruned['bins_ppm'].unique().tolist())
 
def view(formation='', element='', concentration=''):
    
    try:
        fig, ax = plt.subplots()
        pruned[(pruned['formation']==formation) & (pruned['element']==element) &(pruned['bins_ppm']>concentration)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
        huc12.plot(ax=ax, alpha=0.1)
    except:
        print('This combination does not exist')
    


w = widgets.Select(options=formations)
v = widgets.Select(options=elements)
u = widgets.FloatText(value=1)

interact(view, formation=w, element=v, concentration=u)
#interact(view, y=v)

interactive(children=(Select(description='formation', options=('Absaroka Volcanic Supergroup: Hominy Peak Formation', 'Absaroka Volcanic Supergroup: Sunlight Group - Trout Peak Trachyandesite', 'Absaroka Volcanic Supergroup: Sunlight Group - Trout Peak Trachyandesite, Wapiti Formation, Crescent Hill Basalt, and Mount Wallace Formation', 'Absaroka Volcanic Supergroup: Sunlight Group - Wapiti Formation', 'Absaroka Volcanic Supergroup: Thorofare Creek Group - Aycross Formation', 'Absaroka Volcanic Supergroup: Thorofare Creek Group - Tepee Trail Formation', 'Absaroka Volcanic Supergroup: Thorofare Creek Group - Two Ocean and Langford Formations; may include Trout Peak Trachyandesite of Sunlight Group', 'Absaroka Volcanic Supergroup: Thorofare Creek Group - Wiggins Formation', 'Absaroka Volcanic Supergroup: Thorofare Creek and Sunlight Groups - Two Ocean and Langford Formations, Trout Peak Trachyandesite, and Wapiti Formation', 'Absaroka Volcanic Supergroup: Washburn Group - Includes Sepulcher, Lamar River, and Cathedral Cliffs Formations', 'Adaville Formation', 'Alkalic extrusive and intrusive igneous rocks', 'Alkalic intrusive and extrusive igneous rocks', 'Alluvium and colluvium', 'Ankareh Formation, Thaynes Limestone, Woodside Shale, and Dinwoody Formation', 'Aspen Shale', 'Bacon Ridge Sandstone', 'Basalt flows and intrusive igneous rocks', 'Battle Spring Formation', 'Baxter Shale', 'Bear River Formation', 'Bighorn Dolomite', 'Bighorn Dolomite, Gallatin Limestone, and Gros Ventre Formation (TB); Bighorn Dolomite, Snowy Range Formation, Pilgrim Limestone, Park Shale, Meagher Limestone, Wolsey Shale, and Flathead Sandstone (Y); Bighorn Dolomite, Gallatin Limestone, Gros Ventre F', 'Bishop Conglomerate', 'Blind Bull Formation', 'Bridger Formation', 'Brule Member of White River Formation - Locally includes upper conglomerate member', 'Bug Formation (fn 1)', 'Caldwell Canyon Volcanics', 'Camp Davis Formation', 'Carlile Shale', 'Casper Formation', 'Casper and Fountain Formations', 'Cathedral Bluffs Tongue of Wasatch Formation', 'Chadron Member of White River Formation', 'Chugwater Formation (N, NE), or Chugwater Group or Formation (S)', 'Chugwater and Dinwoody Formations', 'Chugwater and Goose Egg Formations', 'Cloverly and Morrison Formations (N, S) or Cloverly Formation (Hartville uplift), or Inyan Kara Group (Black Hills), and Morrison Formation (NE)', 'Cloverly, Morrison, Sundance, and Gypsum Spring Formations', 'Cloverly, Morrison, Sundance, and Gypsum Spring Formations, and Nugget Sandstone', 'Cloverly, Morrison, and Sundance Formations', 'Coalmont Formation', 'Cody Shale', 'Cody Shale and Frontier Formation', 'Colter Formation', 'Conant Creek tuff', 'Conglomerate (fn 1)', 'Conglomerate of Roaring Creek', 'Conglomerate of Sublette Range', 'Crandall Conglomerate', 'Crooks Gap Conglomerate', 'Dacite and quartz latite intrusive and extrusive igneous rocks', 'Devils Basin Formation', 'Dune sand and loess', 'Evanston Formation', 'Everts Formation, Eagle Sandstone, and Telegraph Creek Formation', 'Ferris Formation', 'Forelle Limestone and Satanka Shale', 'Fort Union Formation', 'Fowkes Formation', 'Fox Hills Sandstone', 'Fox Hills Sandstone and Bearpaw Shale', 'Fox Hills Sandstone and Lewis Shale', 'Frontier Formation', 'Frontier Formation, and Mowry and Thermopolis Shales', 'Gallatin Limestone, Gros Ventre Formation and equivalents, and Flathead Sandstone (N), or Cambrian rocks (S)', 'Gannett Group - Includes Smoot Formation, Draney Limestone, Bechler Conglomerate, Peterson Limestone, and Ephraim Conglomerate', 'Glacial deposits', 'Goose Egg Formation', 'Granite gneiss', 'Granitic conglomerate above or in upper part of Wasatch Formation', 'Granitic rocks of 1,700-Ma age group', 'Granitic rocks of 2,000-Ma age group', 'Granitic rocks of 2,600-Ma age group', 'Granodiorite of the Louis Lake pluton', 'Gravel, pediment, and fan deposits', 'Green River and Wasatch Formations', 'Greenhorn Formation and Belle Fourche Shale', 'Greenhorn Form

<function __main__.view>

In [224]:
inHouse=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\inhouse_elemental.csv')

In [238]:
inHouse.iloc[0]['co_ppm']

27.1